https://phdinds-aim.github.io/time_series_handbook/08_WinningestMethods/lightgbm_m5_forecasting.html

https://www.kaggle.com/code/ratan123/m5-forecasting-lightgbm-with-timeseries-splits

### Fine tuning only on 1 level ahead forecast horizon

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [19]:
id_cols = ['Client', 'Warehouse', 'Product']

In [2]:
train_df = pd.read_csv("../data/phase0_train.csv")
test_df = pd.read_csv("../data/phase0_test.csv")

test_df['ds'] = pd.to_datetime(test_df['ds'])
train_df['ds'] = pd.to_datetime(train_df['ds'])

In [3]:
train_df.head()

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2020-07-06,10.900001,7.0
1,0,1,367,2020-07-13,10.900001,7.0
2,0,1,367,2020-07-20,10.900001,7.0
3,0,1,367,2020-07-27,15.582857,7.0
4,0,1,367,2020-08-03,27.289999,7.0


In [4]:
train_df.shape

(2559010, 6)

In [5]:
test_df.shape

(195689, 6)

In [6]:
df = pd.concat([train_df, test_df])

In [7]:
df = df.reset_index(drop=True)

In [8]:
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(by  = ['Client', 'Warehouse', 'Product', 'ds'])

In [9]:
df.head()

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2020-07-06,10.900001,7.0
1,0,1,367,2020-07-13,10.900001,7.0
2,0,1,367,2020-07-20,10.900001,7.0
3,0,1,367,2020-07-27,15.582857,7.0
4,0,1,367,2020-08-03,27.289999,7.0


In [10]:
test_df.tail()

,Client,Warehouse,Product,ds,Price,y
195684,46,318,14294,2023-12-04,NaN,0.0
195685,46,318,14294,2023-12-11,46.990000,1.0
195686,46,318,14294,2023-12-18,46.990000,1.0
195687,46,318,14294,2023-12-25,39.190000,1.0
195688,46,318,14294,2024-01-01,45.423336,3.0


In [11]:
df.tail()

,Client,Warehouse,Product,ds,Price,y
2754694,46,318,14294,2023-12-04,NaN,0.0
2754695,46,318,14294,2023-12-11,46.990000,1.0
2754696,46,318,14294,2023-12-18,46.990000,1.0
2754697,46,318,14294,2023-12-25,39.190000,1.0
2754698,46,318,14294,2024-01-01,45.423336,3.0


In [12]:
id_cols = ['Client', 'Warehouse', 'Product']
for col in id_cols:
    df[col] = df[col].astype('category')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2754699 entries, 0 to 2754698
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Client     category      
 1   Warehouse  category      
 2   Product    category      
 3   ds         datetime64[ns]
 4   Price      float64       
 5   y          float64       
dtypes: category(3), datetime64[ns](1), float64(2)
memory usage: 97.6 MB


In [16]:
def feature_creation(data, prediction_horizon = 13):
    id_cols = ['Client', 'Warehouse', 'Product']
    data['prev_day']= data['ds'] - pd.Timedelta(days=7)

    from pandas.tseries.holiday import USFederalHolidayCalendar
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=data['prev_day'].min(), end=data['ds'].max()).to_pydatetime()
    
    data['holiday_count'] = data.apply(
        lambda x: sum(1 for holiday in holidays if holiday.date() >= x['prev_day'].date() and holiday.date() <= x['ds'].date()), axis=1)
    data['is_holiday'] = data['holiday_count'].apply(lambda x: 1 if x > 0 else 0)

    data['is_prev_1week_holidays'] = data.groupby(id_cols, observed=True)['holiday_count'].transform(lambda x: x.shift(1)) 
    data['is_prev_2week_holidays'] = data.groupby(id_cols, observed=True)['holiday_count'].transform(lambda x: x.shift(2)) 

    data["year"] = data["ds"].dt.year
    data["month"] = data["ds"].dt.month
    data["week"] = data["ds"].dt.isocalendar().week
    data["day"] = data["ds"].dt.day
    data["day_of_week"] = data["ds"].dt.weekday
    data["quarter"] = data["ds"].dt.quarter

    for week in range(prediction_horizon, prediction_horizon+10):#[13,14,15,16,17,18,19]:
        data[f'sales_lag_{week}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(week))
    
    for week in range(1,14): # price is availble for current and past durations
        data[f'price_lag_{week}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.shift(week))
        data[f'price_rolling_mean_{week}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.rolling(week).mean())
        data[f'price_rolling_std_{week}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.rolling(week).std())
    
    for rolling_window in [13,20,30]:
        data[f'sales_rolling_mean_{rolling_window}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(prediction_horizon).rolling(rolling_window).mean())
        data[f'sales_rolling_std_{rolling_window}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(prediction_horizon).rolling(rolling_window).std())

    data = data.drop(columns = ['prev_day'])
    return data    


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2754699 entries, 0 to 2754698
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Client     category      
 1   Warehouse  category      
 2   Product    category      
 3   ds         datetime64[ns]
 4   Price      float64       
 5   y          float64       
dtypes: category(3), datetime64[ns](1), float64(2)
memory usage: 97.6 MB


In [ ]:
for horizon in range(1,2):# 14):
    data = feature_creation(df, prediction_horizon=horizon)
    # Remove rows that exist in df_subset
    
    final_train_df = data.merge(test_df[['Client', 'Warehouse', 'Product', 'ds']], on = ['Client', 'Warehouse', 'Product', 'ds'], how = 'left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
    final_test_df = data.merge(train_df[['Client', 'Warehouse', 'Product', 'ds']], on = ['Client', 'Warehouse', 'Product', 'ds'], how = 'left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

    x = final_test_df.merge(test_df, on = ['Client', 'Warehouse', 'Product', 'ds'], how = 'outer', indicator=True)
    print(x['_merge'].value_counts())
    x = final_train_df.merge(train_df, on = ['Client', 'Warehouse', 'Product', 'ds'], how = 'outer', indicator=True)
    print(x['_merge'].value_counts())

    final_train_df.reset_index(drop=True, inplace=True)
    final_test_df.reset_index(drop=True, inplace=True)

    final_train_df.to_csv(f"train_df_{horizon}.csv", index=False)
    final_test_df.to_csv(f"test_df_{horizon}.csv", index= False)



_merge
both          195689
left_only          0
right_only         0
Name: count, dtype: int64
_merge
both          2559010
left_only           0
right_only          0
Name: count, dtype: int64


In [19]:
test_df.columns

Index(['Client', 'Warehouse', 'Product', 'ds', 'Price', 'y'], dtype='object')

In [ ]:

df_naive = df.copy()

df_naive['y_pred_naive_13'] = df_naive.groupby(id_cols)['y'].transform(lambda x: x.shift(13))
df_naive['y_pred_naive_26'] = df_naive.groupby(id_cols)['y'].transform(lambda x: x.shift(26))
df_naive_test = df_naive.merge(
    test_df[['Client', 'Warehouse', 'Product', 'ds']], 
    on = ['Client', 'Warehouse', 'Product', 'ds'], how = 'inner', indicator=True)

C:\Users\rucha\AppData\Local\Temp\ipykernel_33940\4069501889.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_naive['y_pred_naive_13'] = df_naive.groupby(id_cols)['y'].transform(lambda x: x.shift(13))
C:\Users\rucha\AppData\Local\Temp\ipykernel_33940\4069501889.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_naive['y_pred_naive_26'] = df_naive.groupby(id_cols)['y'].transform(lambda x: x.shift(26))


In [21]:
test_df['ds'].min(),test_df['ds'].max() 

(Timestamp('2023-10-09 00:00:00'), Timestamp('2024-01-01 00:00:00'))

In [22]:
# Custom error metric 
def custom_error(true, pred):
        
    abs_err = np.nansum(np.abs(pred - true))
    err = np.nansum((pred - true))
    score = abs_err + abs(err)
    score /= true.sum().sum()
    return score

In [23]:
cerr_naive13 = custom_error(df_naive['y'], df_naive['y_pred_naive_13'])
cerr_naive26 = custom_error(df_naive['y'], df_naive['y_pred_naive_26'])

print('  s13-Naive MAE: %.4f' % (np.mean(np.abs(df_naive['y_pred_naive_13'] - test_df['y']))))
print(' s26-Naive MAE: %.4f' % (np.mean(np.abs(df_naive['y_pred_naive_26'] - test_df['y']))))

print('Custom error')
print('  s13-Naive MAE: %.4f' % cerr_naive13)
print(' s26-Naive MAE: %.4f' % cerr_naive26)


  s13-Naive MAE: 59.0698
 s26-Naive MAE: 59.2719
Custom error
  s13-Naive MAE: 0.6608
 s26-Naive MAE: 0.7540


### Data prep for final predictions

In [4]:
# Read price data
price_p0 = pd.read_csv("../data/Phase 0 - Price.csv", na_values=np.nan)
price_p0["Value"] = "Price"
price_p0 = price_p0.set_index(["Client", "Warehouse","Product", "Value"]).stack()

# Read sales data
sales_p0 = pd.read_csv("../data/Phase 0 - Sales.csv", na_values=np.nan)
sales_p0["Value"] = "Sales"
sales_p0 = sales_p0.set_index(["Client", "Warehouse","Product", "Value"]).stack()

# Read price validation data
price_p1 = pd.read_csv("../data/Phase 1 - Price.csv", na_values=np.nan)
price_p1["Value"] = "Price"
price_p1 = price_p1.set_index(["Client", "Warehouse","Product", "Value"]).stack()

# Read sales validation data
sales_p1 = pd.read_csv("../data/Phase 1 - Sales.csv", na_values=np.nan)
sales_p1["Value"] = "Sales"
sales_p1 = sales_p1.set_index(["Client", "Warehouse","Product", "Value"]).stack()

# Read sales test data
sales_p2 = pd.read_csv("../data/Phase 2 - Sales.csv", na_values=np.nan)
sales_p2["Value"] = "Sales"
sales_p2 = sales_p2.set_index(["Client", "Warehouse","Product", "Value"]).stack()


In [5]:

# Create single dataframe
df = pd.concat([price_p0, sales_p0]).unstack("Value").reset_index()
df.columns = ["Client", "Warehouse", "Product", "ds", "Price", "y"]
df["ds"] = pd.to_datetime(df["ds"])
df = df.astype({"Price": np.float32,
                "y": np.float32,
                "Client": "category",
                "Warehouse": "category",
                "Product": "category",
                })

df_test = pd.concat([price_p1, sales_p1]).unstack("Value").reset_index()
df_test.columns = ["Client", "Warehouse", "Product", "ds", "Price", "y"]
df_test["ds"] = pd.to_datetime(df_test["ds"])
df_test = df_test.astype({"Price": np.float32,
                "y": np.float32,
                "Client": "category",
                "Warehouse": "category",
                "Product": "category",
                })

In [6]:
df.head()

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2020-07-06,10.900001,7.0
1,0,1,367,2020-07-13,10.900001,7.0
2,0,1,367,2020-07-20,10.900001,7.0
3,0,1,367,2020-07-27,15.582857,7.0
4,0,1,367,2020-08-03,27.289999,7.0


In [7]:
df_test.head()

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2023-10-09,51.860001,1.0
1,0,1,367,2023-10-16,51.860001,1.0
2,0,1,367,2023-10-23,51.860001,1.0
3,0,1,367,2023-10-30,51.230000,2.0
4,0,1,367,2023-11-06,51.230000,1.0


In [11]:
train_df = pd.concat([df, df_test])
train_df.reset_index(drop = True, inplace= True)

In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2754699 entries, 0 to 2754698
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Client     category      
 1   Warehouse  category      
 2   Product    category      
 3   ds         datetime64[ns]
 4   Price      float32       
 5   y          float32       
dtypes: category(3), datetime64[ns](1), float32(2)
memory usage: 55.5 MB


In [17]:
sales_p2 = sales_p2.unstack("Value").reset_index()
sales_p2.columns = ["Client", "Warehouse", "Product", "ds", "y"]

In [18]:
sales_p2['ds'].unique()

array(['2024-01-08', '2024-01-15', '2024-01-22', '2024-01-29',
       '2024-02-05', '2024-02-12', '2024-02-19', '2024-02-26',
       '2024-03-04', '2024-03-11', '2024-03-18', '2024-03-25',
       '2024-04-01'], dtype=object)

In [57]:
train_df.head()

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2020-07-06,10.900001,7.0
1,0,1,367,2020-07-13,10.900001,7.0
2,0,1,367,2020-07-20,10.900001,7.0
3,0,1,367,2020-07-27,15.582857,7.0
4,0,1,367,2020-08-03,27.289999,7.0


In [76]:
combinations = train_df[id_cols].drop_duplicates()
combinations.shape

(15053, 3)

In [77]:
sales_p2['ds'].nunique()

13

In [ ]:
combinations.shape[0] * sales_p2['ds'].nunique()

195689

In [79]:
combinations = combinations.merge(pd.DataFrame({'ds': sales_p2['ds'].unique()}), how='cross')
combinations['Price'] = np.nan
combinations['y'] = np.nan
combinations['ds']= pd.to_datetime(combinations['ds'])
combinations.shape


(195689, 6)

In [80]:
train_df.head()

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2020-07-06,10.900001,7.0
1,0,1,367,2020-07-13,10.900001,7.0
2,0,1,367,2020-07-20,10.900001,7.0
3,0,1,367,2020-07-27,15.582857,7.0
4,0,1,367,2020-08-03,27.289999,7.0


In [ ]:
preprocess_df = pd.concat([train_df, combinations])
preprocess_df['ds'] = pd.to_datetime(preprocess_df['ds'])

C:\Users\rucha\AppData\Local\Temp\ipykernel_5348\2097514771.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  preprocess_df = pd.concat([train_df, combinations])


In [82]:
preprocess_df['ds'].unique()

<DatetimeArray>
['2020-07-06 00:00:00', '2020-07-13 00:00:00', '2020-07-20 00:00:00',
 '2020-07-27 00:00:00', '2020-08-03 00:00:00', '2020-08-10 00:00:00',
 '2020-08-17 00:00:00', '2020-08-24 00:00:00', '2020-08-31 00:00:00',
 '2020-09-07 00:00:00',
 ...
 '2024-01-29 00:00:00', '2024-02-05 00:00:00', '2024-02-12 00:00:00',
 '2024-02-19 00:00:00', '2024-02-26 00:00:00', '2024-03-04 00:00:00',
 '2024-03-11 00:00:00', '2024-03-18 00:00:00', '2024-03-25 00:00:00',
 '2024-04-01 00:00:00']
Length: 196, dtype: datetime64[ns]

In [83]:
preprocess_df = preprocess_df.sort_values(by = id_cols+['ds'])

In [91]:
def feature_creation(data):
    id_cols = ['Client', 'Warehouse', 'Product']
    data['prev_day']= data['ds'] - pd.Timedelta(days=7)

    from pandas.tseries.holiday import USFederalHolidayCalendar
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=data['prev_day'].min(), end=data['ds'].max()).to_pydatetime()
    
    data['holiday_count'] = data.apply(
        lambda x: sum(1 for holiday in holidays if holiday.date() >= x['prev_day'].date() and holiday.date() <= x['ds'].date()), axis=1)
    data['is_holiday'] = data['holiday_count'].apply(lambda x: 1 if x > 0 else 0)

    data['is_prev_1week_holidays'] = data.groupby(id_cols, observed=True)['holiday_count'].transform(lambda x: x.shift(1)) 
    data['is_prev_2week_holidays'] = data.groupby(id_cols, observed=True)['holiday_count'].transform(lambda x: x.shift(2)) 

    data["year"] = data["ds"].dt.year
    data["month"] = data["ds"].dt.month
    data["week"] = data["ds"].dt.isocalendar().week
    data["day"] = data["ds"].dt.day
    data["day_of_week"] = data["ds"].dt.weekday
    data["quarter"] = data["ds"].dt.quarter

    for week in range(1, 14+10):
        data[f'sales_lag_ph_{week}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(week))
        data[f'price_lag__ph_{week}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.shift(week))
    
        for rolling_window in [13,26,39]:
            data[f'sales_rolling_mean_ph_{week}_{rolling_window}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(week).rolling(rolling_window).mean())
            data[f'sales_rolling_std_ph_{week}_{rolling_window}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(week).rolling(rolling_window).std())
            data[f'price_rolling_mean_ph_{week}_{rolling_window}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.shift(week).rolling(rolling_window).mean())
            data[f'price_rolling_std_ph_{week}_{rolling_window}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.shift(week).rolling(rolling_window).std())

    data = data.drop(columns = ['prev_day'])
    return data    


In [92]:
preprocess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2950388 entries, 0 to 195688
Data columns (total 44 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   Client                      category      
 1   Warehouse                   category      
 2   Product                     category      
 3   ds                          datetime64[ns]
 4   Price                       float32       
 5   y                           float32       
 6   prev_day                    datetime64[ns]
 7   holiday_count               int64         
 8   is_holiday                  int64         
 9   is_prev_1week_holidays      float64       
 10  is_prev_2week_holidays      float64       
 11  year                        int32         
 12  month                       int32         
 13  week                        UInt32        
 14  day                         int32         
 15  day_of_week                 int32         
 16  quarter                 

In [93]:
data = feature_creation(preprocess_df)

C:\Users\rucha\AppData\Local\Temp\ipykernel_5348\1955236656.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'price_rolling_std_ph_{week}_{rolling_window}'] = data.groupby(id_cols, observed=True)['Price'].transform(lambda x: x.shift(week).rolling(rolling_window).std())
C:\Users\rucha\AppData\Local\Temp\ipykernel_5348\1955236656.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'sales_lag_ph_{week}'] = data.groupby(id_cols, observed=True)['y'].transform(lambda x: x.shift(week))
C:\Users\rucha\AppData\Local\Temp

In [ ]:
columns = ['holiday_count', 'is_holiday', 'is_prev_1week_holidays', 'is_prev_2week_holidays',
           'year', 'month', 'week', 'day', 'day_of_week', 'quarter']
horizon_specific_cols = ['sales_lag_ph_{i}' for i in range(horizon, 14+10)] +\
                        ['price_lag__ph_{i}' for i in range(horizon, 14+10)] +\
                        ['price_rolling_mean_ph_{i}' for i in range(horizon, 14+10)] +\
                        ['price_rolling_std_ph_{i}' for i in range(horizon, 14+10)] +\
                        ['sales_rolling_std_ph_{i}' for i in range(horizon, 14+10)] +\
                        ['sales_rolling_mean_ph_{i}' for i in range(horizon, 14+10)]

In [95]:
data.head()

,Client,Warehouse,Product,ds,Price,y,holiday_count,is_holiday,is_prev_1week_holidays,is_prev_2week_holidays,...,price_rolling_mean_ph_23_13,price_rolling_std_ph_23_13,sales_rolling_mean_ph_23_26,sales_rolling_std_ph_23_26,price_rolling_mean_ph_23_26,price_rolling_std_ph_23_26,sales_rolling_mean_ph_23_39,sales_rolling_std_ph_23_39,price_rolling_mean_ph_23_39,price_rolling_std_ph_23_39
0,0,1,367,2020-07-06,10.900001,7.0,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,367,2020-07-13,10.900001,7.0,0,0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1,367,2020-07-20,10.900001,7.0,0,0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,367,2020-07-27,15.582857,7.0,0,0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1,367,2020-08-03,27.289999,7.0,0,0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
sales_p2['ds'] = pd.to_datetime(sales_p2['ds'])

In [99]:
train_df = data[~data['ds'].isin(sales_p2['ds'].unique())]
test_df = data[data['ds'].isin(sales_p2['ds'].unique())]

train_df.to_csv("../data/preprocessed_train_df.csv", index = False)
test_df.to_csv("../data/preprocessed_test_df.csv", index = False)

In [100]:
sales_p2.to_csv("../data/actual_sales_p2.csv", index = False)